In [2]:
import subprocess
from itertools import product
from multiprocessing import Pool, Queue
import time
import argparse
import threading

def parse_arguments():
    parser = argparse.ArgumentParser(description="Task Executor for GPU-based Evaluation")
    parser.add_argument("--bench_type", type=str, required=True, help="Benchmark type (e.g., MMLU or TrustLLM)")
    parser.add_argument("--model_type", type=str, required=True, help="Model type (e.g., Kangaroo or Baseline)")
    parser.add_argument("--GPU_number", type=int, required=True, help="Number of GPUs to use")
    parser.add_argument("--interval", type=int, default=20, help="Interval between command executions in seconds")
    return parser.parse_args()

def get_free_gpus():
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'], stdout=subprocess.PIPE)
    gpu_memory = result.stdout.decode('utf-8').strip().split('\n')
    gpu_memory = [(i, int(mem)) for i, mem in enumerate(gpu_memory)]
    sorted_gpus = sorted(gpu_memory, key=lambda x: x[1], reverse=True)
    return [gpu[0] for gpu in sorted_gpus if gpu[1] > 16000]


In [3]:
get_free_gpus()

[3, 5, 7]

In [15]:
import difflib
from termcolor import colored

def compare_sentences(sentences):
    # Ensure we have at least two sentences to compare
    if len(sentences) < 2:
        print("Please provide at least two sentences to compare.")
        return

    # Convert sentences to lists of words
    word_lists = [sentence.split() for sentence in sentences]

    # Initialize a list to store the colored words for each sentence
    colored_sentences = [[] for _ in sentences]

    # Compare each sentence with the first sentence
    for i in range(1, len(sentences)):
        differ = difflib.SequenceMatcher(None, word_lists[0], word_lists[i])
        
        # Process the differences
        for tag, i1, i2, j1, j2 in differ.get_opcodes():
            if tag == 'equal':
                # Words are the same, add them without color
                for k in range(i1, i2):
                    colored_sentences[0].append(word_lists[0][k])
                    colored_sentences[i].append(word_lists[i][j1 + k - i1])
            elif tag == 'replace' or tag == 'delete' or tag == 'insert':
                # Words are different, color them
                color = ['red', 'green', 'blue', 'yellow', 'magenta', 'cyan'][i % 6]
                for k in range(i1, i2):
                    colored_sentences[0].append(colored(word_lists[0][k], color))
                for k in range(j1, j2):
                    colored_sentences[i].append(colored(word_lists[i][k], color))
    model_list = ["baseline", "medusa  ", "kangaroo"]
    # Print the colored sentences
    for i, colored_sentence in enumerate(colored_sentences):
        print(f"{model_list[i]}: {' '.join(colored_sentence)}")

# Example usage
sentences = [
    "Title: \"Hawaiian Adventures: Discovering the Islands' Rich Culture and Natural Wonders\"\n\nIntroduction:\n\nHawaii, the tropical paradise, has always been a dream destination for travelers around the world. With its lush landscapes, pristine beaches, and rich cultural heritage, the Aloha State offers an unforgettable experience. Recently, I embarked on a journey to explore the islands and immerse myself in their unique culture. In this blog post, I'll be sharing my adventures and must-see attractions that made my trip to Hawaii an unforgettable experience.\n\nDay 1: Arrival in Honolulu\n\nUpon arriving in Honolulu, the capital city of Hawaii, I was greeted by the warm sun and the sound of the waves crashing against the shore. After checking into my hotel, I headed straight to Waikiki Beach, one of the most famous beaches in the world. The sun was setting, and the sky was painted in shades of orange and pink, creating a picturesque scene. I spent the evening strolling along the beach, taking in the", 
    # "A Aloha, my fellow travelers! Recently, I embarked on a journey to the tropical paradise of Hawaii, a dream destination for many. With its lush landscapes, pristine beaches, and rich cultural heritage, the Aloha State offered an unforgettable experience.\n\nA As I arrived in Honolulu, the capital city of Hawaii, I was greeted by the warm sun and the sound of the waves crashing against the shore. After checking into my hotel, I headed straight to Waikiki Beach, one of the most famous beaches in the world. The sun was setting, and the sky was painted in shades of orange and pink, creating a picturesque scene.\n\nA I spent the evening strolling along the beach, taking in the sights and sounds of the bustling city. The next day, I was eager to explore the island and immerse myself in its unique culture.\n\nA A must-see attraction on my itinerary was the Pearl Harbor Visitor Center, where I learned about the history of the attack on Pearl Harbor during World War II. The museum was a sobering reminder",
    "Title: \"Hawaiian Adventures: Discovering the Islands' Rich Culture and Natural Wonders\"\n\nIntroduction:\n\nHawaii, the tropical paradise, has always been a dream destination for travelers around the world. With its lush landscapes, pristine beaches, and rich cultural heritage, the Aloha State offers an unforgettable experience. Recently, I embarked on a journey to explore the islands and immerse myself in their unique culture. In this blog post, I'll be sharing my adventures and must-see attractions that made my trip to Hawaii an unforgettable experience.\n\nDay 1: Arrival in Honolulu\n\nUpon arriving in Honolulu, the capital city of Hawaii, I was greeted by the warm sun and the sound of the waves crashing against the shore. After checking into my hotel, I headed straight to Waikiki Beach, one of the most famous beaches in the world. The sun was setting, and the sky was painted in shades of orange and pink, creating a picturesque scene. I spent the evening strolling along the beach, taking in the sights and sounds of the bustling city.\n\nDay 2: Hiking to the Top of Diamond Head\n\nThe next day, I decided to explore the island's natural beauty by hiking to the top of Diamond Head. This iconic volcanic crater is a must-see attraction in Honolulu, and the hike offers breathtaking views of the city and the ocean. The trail was steep, but the reward at the top was worth it. I stood at the summit, taking in the stunning panorama and feeling a sense of accomplishment.\n\nDay 3: Exploring the Polynesian Cultural Center\n\nOn the third day, I visited the Polynesian Cultural Center, a unique attraction that showcases the cultures of Hawaii, Fiji, Samoa, and other Pacific Island nations. I spent the day immersing myself in the rich traditions and customs of these cultures, from learning how to hula dance to watching a fire knife dance performance. The center also offers interactive experiences, such as trying traditional foods and watching a canoe race.\n\nDay 4: Snorkeling with Turtles in Hanauma Bay\n\nThe following day, I went snorkeling in Hanauma Bay, a stunning nature preserve known for its crystal-clear waters and abundance of marine life. As I swam among the colorful fish and coral reefs, I was lucky enough to encounter several green sea turtles. It was", 
    # "A Aloha, my recent trip to Hawaii was an adventure like no other. From the moment I arrived in Honolulu, I was greeted by the warm sun and the sound of the waves crashing against the shore. A After checking into my hotel, I headed straight to Waikiki Beach, one of the most famous beaches in the world. A As I strolled along the beach, taking in the sights and sounds of the bustling city, I couldn't help but feel a sense of excitement for the adventures that lay ahead.\n\nA The next day, I decided to explore the island's natural beauty by hiking to the top of Diamond Head. A The steep trail was challenging, but the reward at the top was worth it. A From the summit, I was able to take in a breathtaking panorama of the city and the ocean. A I felt a sense of accomplishment and pride in having reached the top of this iconic volcanic crater.\n\nA On the third day, I visited the Polynesian Cultural Center, a unique attraction that showcases the cultures of Hawaii, Fiji, Samoa, and other Pacific Island nations. A I spent the day immersing myself in the rich traditions and customs of these cultures, from learning how to hula dance to watching a fire knife dance performance. A The center also offers interactive experiences, such as trying traditional foods and watching a canoe race.\n\nA The following day, I went snorkeling in Hanauma Bay, a stunning nature preserve known for its crystal-clear waters and abundance of marine life. A As I swam among the colorful fish and coral reefs, I was lucky enough to encounter several green sea turtles. A It was an unforgettable experience, and I felt grateful for the opportunity to witness these magnificent creatures in their natural habitat.\n\nA Overall, my trip to Hawaii was an adventure filled with cultural experiences and breathtaking natural wonders. A I would highly recommend visiting these attractions and immersing yourself in the island's rich history and traditions. A A Mahalo for reading, and until next time, A Aloha!",
    "Title: \"Hawaiian Adventures: Discovering the Islands' Rich Culture and Natural Wonders\"\n\nIntroduction:\n\nHawaii, the tropical paradise, has always been a dream destination for travelers around the world. With its lush landscapes, pristine beaches, and rich cultural heritage, the Aloha State offers an unforgettable experience.\n\nRecently, I embarked on a journey to explore the islands of Hawaii and immerse myself in their unique culture and stunning natural beauty. In this blog post, I'll be sharing my adventures and must-see attractions that made my trip unforgettable.\n\nDay 1:\n\n1. Arrival in Honolulu:\n\nAfter a long flight, I arrived in Honolulu, the capital city of Hawaii. I was greeted by the warm sun and the lively energy of the city. My first stop was the iconic Waikiki Beach, where I soaked up the sun and enjoyed the crystal-clear waters.\n\n1. Hanauma Bay:\n\nIn the afternoon, I headed to Hanauma Bay, a", 
    # "A Aloha, my recent trip to Hawaii was an adventure like no other. From the moment I arrived in Honolulu, I was greeted by the warm sun and the lively energy of the city.\n\nA As my first stop, I headed to Waikiki Beach, where I soaked up the sun and enjoyed the crystal-clear waters. It was the perfect way to start my Hawaiian adventure.\n\nA Next, I visited Hanauma Bay, a stunning nature preserve known for its clear waters and diverse marine life. I had the opportunity to snorkel and swim with an array of colorful fish, making it a truly unforgettable experience.\n\nA After a day of exploring Honolulu, I headed to the"
]

compare_sentences(sentences)

baseline: Title: "Hawaiian Adventures: Discovering the Islands' Rich Culture and Natural Wonders" Introduction: Hawaii, the tropical paradise, has always been a dream destination for travelers around the world. With its lush landscapes, pristine beaches, and rich cultural heritage, the Aloha State offers an unforgettable experience. Recently, I embarked on a journey to explore the islands and immerse myself in their unique culture. In this blog post, I'll be sharing my adventures and must-see attractions that made my trip to Hawaii an unforgettable experience. Day 1: Arrival in Honolulu Upon arriving in Honolulu, the capital city of Hawaii, I was greeted by the warm sun and the sound of the waves crashing against the shore. After checking into my hotel, I headed straight to Waikiki Beach, one of the most famous beaches in the world. The sun was setting, and the sky was painted in shades of orange and pink, creating a picturesque scene. I spent the evening strolling along the beach, tak

In [2]:
!pip install termcolor

In [13]:
import difflib
from termcolor import colored

def compare_sentences(sentences):
    # Ensure we have at least two sentences to compare
    if len(sentences) < 2:
        print("Please provide at least two sentences to compare.")
        return

    # Convert sentences to lists of words
    word_lists = [sentence.split() for sentence in sentences]

    # Initialize a list to store the colored words for each sentence
    colored_sentences = [[] for _ in sentences]

    # Compare each sentence with the first sentence
    for i in range(1, len(sentences)):
        differ = difflib.SequenceMatcher(None, word_lists[0], word_lists[i])
        
        # Process the differences
        for tag, i1, i2, j1, j2 in differ.get_opcodes():
            if tag == 'equal':
                # Words are the same, add them without color
                for k in range(i1, i2):
                    colored_sentences[0].append(word_lists[0][k])
                    colored_sentences[i].append(word_lists[i][j1 + k - i1])
            elif tag == 'replace' or tag == 'delete' or tag == 'insert':
                # Words are different, color them
                color = ['red', 'green', 'blue', 'yellow', 'magenta', 'cyan'][i % 6]
                for k in range(i1, i2):
                    colored_sentences[0].append(colored(word_lists[0][k], color))
                for k in range(j1, j2):
                    colored_sentences[i].append(colored(word_lists[i][k], color))

    # Print the colored sentences
    for i, colored_sentence in enumerate(colored_sentences):
        print(f"Sentence {i + 1}: {' '.join(colored_sentence)}")

# Example usage
sentences = [
    "The quick brown fox jumps over the lazy dog",
    "The fast brown fox leaps over the sleepy dog",
    "A quick brown fox jumps over the lazy cat"
]

compare_sentences(sentences)

Sentence 1: The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog
Sentence 2: The fast brown fox leaps over the sleepy dog
Sentence 3: A quick brown fox jumps over the lazy cat
